<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_train_GPT_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter
## (1) Finetuning de BETO binario

**Natalia Dedandi**


Esta notebook presenta el entrenamiento de un clasificador BETO (BERT en español):

https://github.com/dccuchile/beto?tab=readme-ov-file

Nombre del modelo: dccuchile/bert-base-spanish-wwm-uncased

Para el finetuning se trabaja a partir de un dataset etiquetado en castellano argentino elaborado en el marco del proyecto PÍUBAMAS. Más infomación aqui:
https://huggingface.co/piuba-bigdata



In [ ]:
!pip install datasets seaborn
! pip install -U accelerate
! pip install -U transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# no lograba bajar el dataset porque
# me fallaba bajar archivos de hugginface y tenia que actualizar esto (JUAN MA me ayudo)
#pip install -U datasets

In [ ]:
from datasets import load_dataset
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

(1) Dataset de etiquetado

https://huggingface.co/datasets/piuba-bigdata/contextualized_hate_speech

Este es el dataset original de etiquetado. Contiene los siguientes campos:

ID : identificador

TITLE: título de la noticia

TEXT: texto del tweet del comentario al Tweet original

CONTEXT: tweet original

HATEFUL: 1 hateful, 0 not hateful

BODY: texto de la noticia

CALLS: si se realizó o no un llamado a la acción

WOMEN: discriminación hacia mujeres

LGBTI: discriminación hacia grupos LGBTI

RACISM: discriminación asociada a racismo

CLASS: discriminación vinculada a pobreza, villas

POLITICS: odio o violencia vinculada a las ideas políticas

DISABLED: discriminación a las personas con discapacidad

APPEARENCE: discriminación en base al aspecto físico (incluye especialmente la gordofobia)

CRIMINAL: discriminación hacia personas en el sistema penal o similares



In [ ]:
from datasets import load_dataset

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

El dataset 1 contiene 56869 tw etiquetados.
El dataset se dividió en 36420 de train, test:11343 y vaLidacion:9106

In [ ]:
ds1_train=ds1["train"]
ds1_train

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 36420
})

**Para el entrenamiento se usaron los siguientes tutoriales y documentos:**

Uso  de base el siguiente tutorial para hacer el finetuning:
https://huggingface.co/blog/sentiment-analysis-python
y este:
https://huggingface.co/docs/transformers/training


La idea es utilizar la librería tranbsformers de huggingface para hacerle finetuning a un BETO original, es decir la version de BERT pre-entrenada con español de GOOGLE pero que no haya sido reentrenado con ningún otro dataset.
Aqui para usar transformers de hugginface
https://github.com/huggingface/transformers?tab=readme-ov-file



In [ ]:
import torch
torch.cuda.is_available()

True

### 1. pre-process data

In [ ]:
 ds1['train'][0]

{'id': 343726,
 'title': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad',
 'text': '@usuario Uno menos',
 'context_tweet': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD',
 'HATEFUL': 1,
 'body': 'Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los "festejos" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales \u200bmuestra a Sari llegando a su casa en un auto azul rodeado de tiros de "celebración". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido

In [ ]:
##ME quedo solo con las columnas que necesito para la clasificación binaria
ds_train_small= ds1['train'].select_columns(["text","HATEFUL"])
ds_test_small= ds1['test'].select_columns(["text","HATEFUL"])
ds_val_small= ds1['dev'].select_columns(["text","HATEFUL"])

In [ ]:
##aca les coloco el mismo nombre que espera el modelo (me parece cualquiera pero no logre que ande sino)
ds_train_small=ds_train_small.rename_column("HATEFUL", "label")
ds_test_small=ds_test_small.rename_column("HATEFUL", "label")
ds_val_small=ds_val_small.rename_column("HATEFUL", "label")

In [ ]:
ds_train_small

Dataset({
    features: ['text', 'label'],
    num_rows: 36420
})

In [ ]:

from transformers import BertForMaskedLM, BertTokenizer

Utiizo las librerías de transformers para preparar los datos de entrenamiento

Lo primero es tokenizar en función del modelo seleccionado, en nuestro caso BETO

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

Le aplico la función tokenizer del modelo BETO a mis conjuntos, el de entrenamiento, el de test y el de validación

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = ds_train_small.map(preprocess_function, batched=True)
tokenized_test = ds_test_small.map(preprocess_function, batched=True)
tokenized_val = ds_val_small.map(preprocess_function, batched=True)

Map:   0%|          | 0/36420 [00:00<?, ? examples/s]

Map:   0%|          | 0/11343 [00:00<?, ? examples/s]

Map:   0%|          | 0/9106 [00:00<?, ? examples/s]

In [ ]:
tokenized_train[0]

{'text': '@usuario Uno menos',
 'label': 1,
 'input_ids': [4, 985, 8563, 1614, 1885, 5],
 'token_type_ids': [0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

La siguiente función (según el ejemplo) transforma los datos de entrenamiento agregandole el padding

Se usará luego en el Trainer

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2 Entreno el modelo.

Voy a entrenar el modelo con el TRAINER de transformers. Instancio el modelo pre-entrenado BETO y aca selecciono 2 labels ya que se hace una clasificación binaria

Modelo: dccuchile/bert-base-spanish-wwm-uncased


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
##FUNCION PARA COMPUTAR LAS METRIcas

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score,confusion_matrix,recall_score
from transformers import EvalPrediction

def compute_metrics(eval_pred):

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    recall = recall_score(predictions, labels)
    accuracy = accuracy_score(predictions, labels)
    f1 = f1_score(predictions, labels)
    auc = roc_auc_score(predictions, labels)
    cm = confusion_matrix(predictions, labels)

    return { "cm": cm, "recall": recall,  "accuracy": accuracy,"f1": f1, "auc":auc}



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Revision de parametros de TRAINER
https://huggingface.co/docs/transformers/main_classes/trainer

Definicipon de parametros
https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/trainer#transformers.TrainingArguments



In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "beto_hate_speech_ar_bin"

training_args = TrainingArguments(
	 output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
# Train pre-trained model
trainer.train()

Step,Training Loss
500,0.020500
1000,0.033200
1500,0.048800
2000,0.044800
2500,0.045200
3000,0.042600
3500,0.043300
4000,0.053700
4500,0.098100


TrainOutput(global_step=4554, training_loss=0.04899979433470983, metrics={'train_runtime': 953.0179, 'train_samples_per_second': 76.431, 'train_steps_per_second': 4.779, 'total_flos': 2173085595096240.0, 'train_loss': 0.04899979433470983, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "[[8971  753]
 [ 575 1044]]" of type <class 'numpy.ndarray'> for key "eval/cm" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.7231101393699646,
 'eval_cm': array([[8971,  753],
        [ 575, 1044]]),
 'eval_recall': 0.6448424953675108,
 'eval_accuracy': 0.8829233888741955,
 'eval_f1': 0.6112412177985949,
 'eval_auc': 0.7837026133768857,
 'eval_runtime': 38.2816,
 'eval_samples_per_second': 296.304,
 'eval_steps_per_second': 18.521,
 'epoch': 2.0}

Voy a probar el modelo entrenado y guardado

In [ ]:
from transformers import pipeline
data = ["hay que matarlos a todos", "Vamos a la plaza a marchar"]
specific_model = pipeline(model="natdebandi/beto_hate_speech_ar_bin")
specific_model(data)

config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/735k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.9985326528549194},
 {'label': 'LABEL_0', 'score': 0.9996645450592041}]